In [ ]:
import geopandas as gpd

In [2]:
# Load your shapefile (replace with your actual file path)
gdf = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/populaion_commune/populaion_commune.shp")

In [3]:
# Preview columns to find the commune name column
print(gdf.columns)

Index(['Code_Commu', 'Code_Provi', 'CODE_REGIO', 'nom', 'Marocains_',
       'Etrangers_', 'Populati_1', 'Menages_', 'nom_arabe', 'geometry'],
      dtype='object')


In [5]:
# Assuming the commune name is in a column called 'NAME'
# You can adjust this depending on the actual column name
gdf['centroid'] = gdf.geometry.centroid

C:\Users\HP\AppData\Local\Temp\ipykernel_24688\2328595073.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [26]:
# Extract relevant data
communes = gdf[['nom', 'centroid']].copy()
communes['latitude'] = communes.centroid.y
communes['longitude'] = communes.centroid.x

In [27]:
# Drop centroid column if not needed
communes = communes.drop(columns='centroid')

In [21]:
# Print or save
print(communes.head())
len(communes)

                               nom   latitude  longitude
0       Commune de Laqsabi Tagoust  28.967768 -10.240091
1           Commune de Rass Oumlil  28.662261 -10.798347
2  Commune de Sidi Ahmed Laaroussi  26.980072 -11.991460
3             Commune de Tizounine  29.099425  -8.302945
4           Commune de Fam El Hisn  28.794484  -8.814432


1505

In [ ]:
# Optionally save to CSV
communes.to_csv("morocco_communes_latlon.csv", index=False)

### Province Evaluation

# Rainfall Data from NASA Power API. 
# For specific locations.
# From 2010 to 2024

In [111]:
import requests

In [ ]:
monthly_url = "https://power.larc.nasa.gov/api/temporal/monthly/point"

In [110]:
commune_rain_data = {}

In [ ]:
for index in range(0, len(communes.latitude)):

    monthly_params = {
    "parameters": "PRECTOTCORR",
    "community": "AG",
    "longitude": communes.longitude[index],
    "latitude": communes.latitude[index],
    "start": "2010",
    "end": "2024",
    "format": "JSON"
    }

    monthly_response = requests.get(monthly_url, params=monthly_params)
    # wait for 5 seconds
    print('sleeping')

    if monthly_response.status_code == 200:
        monthly_data = monthly_response.json()['properties']['parameter']['PRECTOTCORR']
        count = 0
        sum = 0
        yearly_averages = []
        for val in monthly_data.items():
            if count <= 12:
                sum += val[1]
                count+=1
                continue
            yearly_averages.append(sum/count)
            count = 0
            sum = 0
        commune_rain_data[communes.nom[index]] = yearly_averages
    else:
        print("error fetching data")
        commune_rain_data[communes.nom[index]] = monthly_response.headers.get("title")       

In [126]:
# len(commune_rain_data) - len(communes.latitude)
commune_rain_data[communes.nom[2]]

[0.2784615384615385,
 0.35923076923076913,
 0.24307692307692305,
 0.18846153846153849,
 0.486923076923077,
 0.8007692307692308,
 0.6461538461538462,
 0.43769230769230766,
 0.3215384615384615,
 0.2684615384615384,
 0.8238461538461538,
 0.32076923076923075,
 0.7892307692307694]

In [133]:
len(communes.latitude)
communes.nom[""]

1505

In [121]:
((1 * 1505) / 60 / 60 ) * 60

25.083333333333332

In [ ]:
count = 0
sum = 0
yearly_averages = []
for val in monthly_data.items():
    if count <= 12:
        sum += val[1]
        count+=1
        continue
    yearly_averages.append(sum/count)
    count = 0
    sum = 0


In [59]:
monthly_data.items()

dict_items([('202001', 0.17), ('202002', 0.01), ('202003', 0.9), ('202004', 0.08), ('202005', 1.45), ('202006', 0.11), ('202007', 0.55), ('202008', 0.02), ('202009', 0.04), ('202010', 0.15), ('202011', 2.23), ('202012', 0.24), ('202013', 0.5), ('202101', 1.26), ('202102', 1.51), ('202103', 0.32), ('202104', 0.12), ('202105', 0.03), ('202106', 0.04), ('202107', 0.08), ('202108', 0.07), ('202109', 0.04), ('202110', 0.01), ('202111', 0.54), ('202112', 0.08), ('202113', 0.34), ('202201', 0.07), ('202202', 0.05), ('202203', 1.81), ('202204', 0.43), ('202205', 0.43), ('202206', 0.09), ('202207', 0.03), ('202208', 0.15), ('202209', 0.71), ('202210', 0.08), ('202211', 1.61), ('202212', 6.75), ('202213', 1.02), ('202301', 0.09), ('202302', 2.99), ('202303', 0.07), ('202304', 0.33), ('202305', 0.2), ('202306', 4.56), ('202307', 0.01), ('202308', 0.01), ('202309', 0.07), ('202310', 0.64), ('202311', 0.08), ('202312', 0.36), ('202313', 0.78), ('202401', 0.01), ('202402', 0.34), ('202403', 0.65), (

In [53]:
# Calculate averages
yearly_averages = {year: yearly_totals[year] / yearly_counts[year] for year in yearly_totals}

In [55]:
yearly_averages

{'2020': 0.4961538461538462,
 '2021': 0.3415384615384615,
 '2022': 1.0176923076923077,
 '2023': 0.7838461538461539,
 '2024': 0.27384615384615385}